In [49]:
import requests
from pprint import pprint
import pandas as pd
import json
from gensim import corpora, models, similarities

In [47]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [31]:
url = ("http://localhost:8082/api/populatedTags");
response = requests.get(url)
data = json.loads(response.text)
pprint(data)

[{'__v': 0,
  '_chunks': [{'_id': '56aba31c9b1c17c8c853b420',
               'ad_id': 4227,
               'chunk_id': 88795,
               'content': 'Key reponsibilities and tasks'},
              {'_id': '56aba31c9b1c17c8c853b421',
               'ad_id': 4227,
               'chunk_id': 88796,
               'content': 'Development of profitable customer solutions, with '
                          'a holistic view on the customer needs and services '
                          'delivered by Nets'},
              {'_id': '56aba31c9b1c17c8c853b422',
               'ad_id': 4227,
               'chunk_id': 88797,
               'content': 'Participate in customer pre-study projects and RFP '
                          'processes'}],
  '_id': '56af985e9b1c17c8c854751f',
  'content': 'requirements',
  'updated': '2016-02-01T17:39:42.543Z'}]


In [89]:
class TagCorpus(object):
    def __iter__(self):
        for tag in data:
            words = ""
            for chunk in tag['_chunks']:
                words += " " + str(chunk['content']);
            yield words.split()

In [90]:
corpus = TagCorpus() # doesn't load the corpus into memory!
print(corpus)

In [93]:
for vector in corpus: # load one vector into memory at a time
    print(vector)

['Key', 'reponsibilities', 'and', 'tasks', 'Development', 'of', 'profitable', 'customer', 'solutions,', 'with', 'a', 'holistic', 'view', 'on', 'the', 'customer', 'needs', 'and', 'services', 'delivered', 'by', 'Nets', 'Participate', 'in', 'customer', 'pre-study', 'projects', 'and', 'RFP', 'processes']


In [94]:
stoplist = set('for a of the and to in'.split())
# collect statistics about all tokens
dictionary = corpora.Dictionary(vector for vector in corpus)
# remove stop words and words that appear only once
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist
            if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items() if docfreq == 1]

# dictionary.filter_tokens(stop_ids + once_ids) # remove stop words and words that appear only once
dictionary.filter_tokens(stop_ids)

dictionary.compactify() # remove gaps in id sequence after words that were removed
print(dictionary)

Dictionary(21 unique tokens: ['Key', 'reponsibilities', 'services', 'customer', 'RFP']...)
